# Read and Process PCD

In [ ]:
import open3d as o3d


def iterate_pcd_file(file_path):
    pcd = o3d.io.read_point_cloud(file_path)

    # Iterate over each point
    for x, y, z in pcd.points:
        break
        # print(f"Point: ({x}, {y}, {z})")


# Provide the path to your PCD file
PCD_FILEPATH = "/Volumes/mos/data/deepforestry/labled/rev7/2024-01-31-06-28-36_filtered_labeled.pcd"
iterate_pcd_file(PCD_FILEPATH)

# Understand KITTI

In [ ]:
import pathlib
import subprocess
import os

KITTI_API_DIR = pathlib.Path.cwd().parent.parent / "tools/semantic-kitti-api"
DATA_PATH = (
    pathlib.Path(os.environ["DATA_PATH"]) / "kitti/SemanticKITTI/symlinks/motionbev"
)
SEQUENCE = 8

command = [
    "python",
    "visualize.py",
    "--dataset",
    DATA_PATH.as_posix(),
    "--sequence",
    str(SEQUENCE),
]

print(" ".join(command))

#

In [ ]:
subprocess.run(command, cwd=KITTI_API_DIR)

## LiDAR data
It seems like the velodyne .bin format is just [x,y,z,remission], and can be read by `np.fromfile(filename, dtype=np.float32)`, then reshaped with `reshape((-1,4))`

In [ ]:
import yaml
import pathlib

CONFIG_PATH = pathlib.Path.cwd() / "config" / "semantic-kitti.yaml"


def load_config(config_path):
    with open(CONFIG_PATH, "r") as file:
        config = yaml.safe_load(file)
        return config


config = load_config(CONFIG_PATH)


def load_color_map(config):
    color_map = config["color_map"]
    learning_map_inv = config["learning_map_inv"]
    learning_map = config["learning_map"]

    return {
        key: color_map[learning_map_inv[learning_map[key]]]
        for key, value in color_map.items()
    }


color_map = load_color_map(config)

In [ ]:
import os
import pathlib
import time

import numpy as np

RATE_HZ = 10
SEQUENCE = 8
BASE_PATH = pathlib.Path(os.environ["DATA_PATH"])
SEQUENCE_PATH = "kitti/SemanticKITTI/symlinks/motionbev/sequences"
BIN_PATH = BASE_PATH / SEQUENCE_PATH / f"{SEQUENCE:02}/velodyne"
LABEL_PATH = BASE_PATH / SEQUENCE_PATH / f"{SEQUENCE:02}/labels"
CONFIG_PATH = (
    "/Users/ilja/Div/github/testing/scratchpad/notebooks/config/semantic-kitti-mos.yaml"
)


def load_config(config_path):
    with open(CONFIG_PATH, "r") as file:
        config = yaml.safe_load(file)
        return config


def load_color_map(config, return_rgb=False):
    color_map = config["color_map"]
    learning_map_inv = config["learning_map_inv"]
    learning_map = config["learning_map"]

    bgr_color_map = {
        key: color_map[learning_map_inv[learning_map[key]]]
        for key, _ in color_map.items()
    }

    if return_rgb:
        return {key: tuple(reversed(color)) for key, color in bgr_color_map.items()}

    return bgr_color_map


def load_labels(path):
    labels = np.fromfile(path, dtype=np.int32)
    semantic_labels = labels & 0xFFFF
    instance_labels = labels >> 16
    return semantic_labels, instance_labels


def load_scan(path):
    return np.fromfile(path, dtype=np.float32).reshape((-1, 4))


def update_point_cloud(pcd, scan, colors):
    pcd.points = o3d.utility.Vector3dVector(scan[:, :3])
    pcd.colors = o3d.utility.Vector3dVector(colors)


def create_colors(semantic_labels, color_map):
    return (
        np.array([color_map[label] for label in semantic_labels], dtype=np.float64)
        / 255
    )


def update_visualizer(pcd, visualizer):
    visualizer.update_geometry(pcd)
    visualizer.poll_events()
    visualizer.update_renderer()


def combine_labels(semantic_labels, instance_labels):
    labels = (instance_labels << 16) | semantic_labels
    return labels


def main(visualizer):
    first_scan_path, *scan_paths = sorted(BIN_PATH.glob("*.bin"))
    first_label_path, *label_paths = sorted(LABEL_PATH.glob("*.label"))

    config = load_config(CONFIG_PATH)
    color_map = load_color_map(config, return_rgb=True)
    first_scan = load_scan(first_scan_path)
    semantic_labels, _ = load_labels(first_label_path)

    colors = create_colors(semantic_labels, color_map)

    pcd = o3d.geometry.PointCloud()

    update_point_cloud(pcd, first_scan, colors)

    visualizer.create_window()
    visualizer.add_geometry(pcd)

    for scan_path, lable_path in zip(scan_paths, label_paths):
        scan = load_scan(scan_path)
        semantic_labels, _ = load_labels(lable_path)
        colors = create_colors(semantic_labels, color_map)

        update_point_cloud(pcd, scan, colors)

        if pcd.has_colors():
            update_visualizer(pcd, visualizer)

        time.sleep(1 / RATE_HZ)

    visualizer.destroy_window()


visualizer = o3d.visualization.Visualizer()

try:
    main(visualizer)
except KeyboardInterrupt:
    visualizer.destroy_window()
    print("Visualizer closed")